In [1]:
import numpy as np
from scipy.interpolate import interp1d
from parabolic2D import parabolic2D

global eps0, e, p, NE0, E0, F0, U1, U2, mue, mui, alpha, ne_0, N, M, r, hr, hz, R1, R2, R3, R4, R5, D
eps0 = (8.85e-12)
e = (1.6e-19)
p = 1e5
NE0 = 1e18
E0 = 5e3
F0 = 1500
U1 = -100
U2 = 0
mue = 1
mui = 1
alpha = 1
ne_0 = 1e18
k = 1.380649e-23
T = 300

N = 10
M = N

ne = np.zeros((N,M))
De = np.zeros((N,M))

#Di = 
#Dm = 

L = 1
R = 1
r1 = R
hr = (1/(N-0.5))
hz = (L/(M-1))
tEnd = 1
tau = 1

r = np.linspace(hr/2,R,N)

z = np.linspace(0,L,M)

FF = np.zeros(N,M)
Net = np.zeros((N,M))
for j in range(0,M):
    for i in range(0,N):
        FF[i,j] = (-50*z[j]**2)
        Net[i,j] = p/(k*T)
        ne[i][j] = ne_0*(1-(r[i]/R)**2)*(z[j]*(L-z[j]))
ni = ne.copy()
nm = ne.copy()

for i in range(N):
    FF[i,0] = 0 #низ
for i in range(N):
    FF[i,M-1] = -50

In [4]:
Di = np.zeros((N,2))
R1 = np.zeros((N,2))
patternD = 'Diffusion coefficient'
patternR = 'C1    Ar    Effective (momentum)'
with open('output.dat', 'r') as file:
    for index, line in enumerate(file):
        if patternD in line:
            indexD = index
        elif patternR in line:
            indexR = index
            break
file = open('output.dat', 'r')
lines = file.readlines()
for i in range(N):
    Di[i] = [float(lines[indexD+1+i][:5]),float(lines[indexD+1+i][-11:-1])]
    R1[i] = [float(lines[indexR+2+i][:5]),float(lines[indexR+2+i][-11:-1])]
file.close()

R1_linear = interp1d(list(i[0] for i in R1), list(i[1] for i in R1), kind = 'linear')
Di_linear = interp1d(list(i[0] for i in Di), list(i[1] for i in Di), kind = 'linear')

In [5]:

R2 = 6.2e-10
R3 = 2e-7*np.exp(-6.2/((T*k)/e))
R4 = 2.7e-13*((T*k)/e)**(-3/4)
R5 = 8.75e-39*((T*k)/e)**(-9/2)

In [8]:
def EEEE(phi):
    r = np.linspace(hr/2,R,N)
    n1 = len(r)
    n2 = len(z)
    h1 = r[1]-r[0]
    h2 = z[1]-z[0]
    E_r = np.ones((n1, n2), 'float')*10.
    E_z = np.ones((n1, n2), 'float')*10.
    for j in range(0,n2-1):
        E_r[0,j] = (phi[1,j]-phi[0,j])/h1
        for i in range(0,n1-1):
            E_r[i,j] = (phi[i,j]-phi[i+1,j])/h1
    for i in range(0,n1-1):
        E_z[i,0] = (phi[i,1]-phi[i,0])/h2
        for j in range(0,n2-1):
            E_z[i,j] = (phi[i,j]-phi[i,j+1])/h2
    return 

In [9]:
def Rg_pr(FF):
    f = np.zeros((N,M))
    for i in range(N):
        for j in range(M):
            variable_phi=FF[i,j]/Net[i,j]
            if variable_phi < 1: variable_phi = 1
            f[i,j] = R1_linear(float(variable_phi))*ne[i,j]*Net[i,j] + R2*nm[i,j]**2 + R3*nm[i,j]*ne[i,j] - R4*ne[i,j]*ni[i,j] - R5*ne[i,j]**2*ni[i,j]

In [10]:
def fsolve(FF,ne,ni,omega_relax = 1.8):
    FF1 = FF.copy()
    f = e/eps0*(ni-ne)#np.ones(N,M) * (100)
    d = np.ones(1,N*M) #формируем диагональ матрицы СЛАУ
    l1 = np.zeros(1,N*M) 
    l2 = np.zeros(1,N*M)
    for j in range(1,M-1):
        for i in range(1,N-1):
            k = i+j*N
            d[k] = 2/hr**2 +2/hz**2
            l1[k] = -(r[i]-hr/2)/(r[i]*hr**2)
            l2[k] = -1/hz**2
        l1[int((j+1)*N-1)] = -1/hr**2

    tol = (1e-2)
    rn_finish = (1e32)

    kiter = 0
    N_iter = 5

    for p in range(N_iter):
        norm_dif = 0
        rn = 0
        print('rn = ',float(rn))

        print("Расчет")
      
        resid = np.zeros(1,N*M)
        corrector = np.zeros(1,N*M)


        for j in range(1,M-1):
            # Граничные условия второго рода слева  
            rr = (FF[0,j]*(2/hr**2) - FF[1,j]*(2/hr**2))\
                - 1/hz**2 * FF[0,j-1] + 2/hz**2 * FF[0,j] - 1/hz**2 * FF[0,j+1]\
                - f[0,j]
            resid[j*N] = rr
            rn = rn+rr**2
            print('rr невязка левые граница', rr )
            FF1[0,j] = FF[0,j] - (omega_relax*rr)/d[j*N]
            norm_dif = max(norm_dif,abs((omega_relax*rr)/(d[j*N])))
            
            # Обходим внутренние узлы
            for i in range(1,N-1):
                rr = -(FF[i-1,j]*(r[i]- hr/2) -2*FF[i,j]*r[i] + FF[i+1,j]*(r[i]+hr/2)) / (hr**2 * r[i]) -(FF[i,j-1] - 2*FF[i,j] + FF[i,j+1])/hz**2 - f[i,j]
                corrector[i+j*N] = rr - omega_relax*(l1[i+j*N-1]*corrector[i+j*N -1] 
                                                     - l2[i+(j-1)*N]*corrector[i+(j-1)*N])/d[i+j*N]
              
                resid[i+j*N] = rr
                rn = rn+rr**2
                print ('i = ',i,'j = ', j, 'rr = ',float(rr),'rn = ',float(rn))
                FF1[i,j] = FF[i,j] - omega_relax*corrector[i+j*N]
                norm_dif = max(norm_dif,abs(omega_relax*corrector[i+j*N]))

            # справа
            rr = (FF[N-1,j] - FF[N-2,j])*2 / hr**2 - f[N-1,j]\
                - 1/hz**2 * (FF[N-1,j-1] - 2*FF[N-1,j] + FF[N-1,j+1])
            corrector[(j+1)*N-1] = rr - omega_relax*(l1[(j+1)*N-2]*corrector[(j+1)*N-2] 
                                                    - l2[j*N-1]*corrector[j*N-1])/d[(j+1)*N-1]
            print('невязка правой границы',float(rr),'\n')
            resid[(j+1)*N-1] = rr
            rn = rn+rr**2
            FF1[N-1,j] = FF[N-1,j] - (omega_relax*corrector[(j+1)*N-1])
            norm_dif = max(norm_dif,abs(omega_relax*corrector[(j+1)*N-1]))
      
        ddd=np.max(np.abs(FF-FF1))
        try:
            err_c = norm_dif/ddd
        except:
            err_c = 0
        FF=FF1.copy()
        rn = rn*hr*hz
        kiter = p
        rn_finish = rn
        #print("итерация номер ", k, " невязка = ", rn," погрешность = ", err_c)
        if rn_finish < tol**2:
            break
      
    print("число итераций =  ", kiter+1)
    print("невязка приближенного решения",rn)
    print("относительная разность, norm_dif = ", norm_dif)
    return FF,rn

In [11]:
print('решение')
t = 0
while t<= tEnd:
    
    FF,nv = fsolve(FF,ne,ni)
    ne = parabolic2D(r,z,Rg_pr(FF),ne,-1,mue,FF,Di,tau)
    ni = parabolic2D(r,z,Rg_pr(FF),ni,1,mui,FF,Di,tau)
    #nm = parabolic2D(r,z,_,nm,0,mui,FF,Di,tau)
    t = tau+t
    

print((FF.transpose()))


решение
rn =  0.0
Расчет
rr невязка левые граница 0
i =  1 j =  1 rr =  0.0 rn =  0.0
i =  2 j =  1 rr =  0.0 rn =  0.0
i =  3 j =  1 rr =  0.0 rn =  0.0
i =  4 j =  1 rr =  1.4210854715202004e-14 rn =  2.0194839173657902e-28
i =  5 j =  1 rr =  1.4210854715202004e-14 rn =  4.0389678347315804e-28
i =  6 j =  1 rr =  1.4210854715202004e-14 rn =  6.058451752097371e-28
i =  7 j =  1 rr =  0.0 rn =  6.058451752097371e-28
i =  8 j =  1 rr =  -1.4210854715202004e-14 rn =  8.077935669463161e-28
невязка правой границы 0.0 

rr невязка левые граница 0
i =  1 j =  2 rr =  1.4210854715202004e-14 rn =  1.0097419586828951e-27
i =  2 j =  2 rr =  1.4210854715202004e-14 rn =  1.2116903504194741e-27
i =  3 j =  2 rr =  1.4210854715202004e-14 rn =  1.4136387421560532e-27
i =  4 j =  2 rr =  5.684341886080802e-14 rn =  4.6448130099413175e-27
i =  5 j =  2 rr =  4.263256414560601e-14 rn =  6.462348535570529e-27
i =  6 j =  2 rr =  4.263256414560601e-14 rn =  8.27988406119974e-27
i =  7 j =  2 rr =  1.421

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''